In [5]:
import os
import pandas as pd 
import numpy as np
import tensorflow 
#import cv2 #images
# import PIL as pil (Python Image Library? over using PostgreSQL)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm # progress bars
from keras import regularizers

## import dependencies for model
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical, normalize
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

from sklearn.metrics import classification_report, confusion_matrix

In [8]:
# Define paths for the dataset directories
benign_dir = 'SKIN-CANCER-DETECTION-PROJECT-4/train_cancer/benign'
malignant_dir = 'SKIN-CANCER-DETECTION-PROJECT-4/train_cancer/malignant'


In [9]:
# Collect image paths and labels for PostgreSQL upload
data = []
for category, folder in [('benign', benign_dir), ('malignant', malignant_dir)]:
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            file_path = os.path.join(folder, filename)
            data.append({'file_path': file_path, 'label': category})


FileNotFoundError: [Errno 2] No such file or directory: 'SKIN-CANCER-DETECTION-PROJECT-4/train_cancer/benign'

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data)


In [ ]:
# Display the DataFrame to verify the paths and labels
display(df.head())

In [ ]:
# Step: Upload image metadata to PostgreSQL
try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    )
    cursor = connection.cursor()


In [ ]:
    # Create a table for storing image metadata
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS skin_cancer_images (
        id SERIAL PRIMARY KEY,
        file_path TEXT,
        label TEXT
    );
    '''
    cursor.execute(create_table_query)
    connection.commit()

In [ ]:
 # Define the database connection URL
    db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}"
    engine = create_engine(db_url)

In [ ]:
    # Upload the DataFrame to PostgreSQL
    df.to_sql('skin_cancer_images', engine, if_exists='append', index=False)

    print("Data uploaded to PostgreSQL successfully.")
except Exception as e:
    print(f"Error uploading data to PostgreSQL: {e}")
finally:
    # Close the connection
    connection.close()

In [ ]:
df = pd.read_csv('path_to_dataset.csv')
x = df.drop('label',axis=1)
y = df['label']
x = x.to_numpy()
x = x/255
y = to_categorical(y)

In [ ]:
# #labeling manually because there are only 7
label = {
    'Actinic keratoses':0,
    'Basal cell carcinoma':1,
    'Benign keratosis-like lesions':2,
    'Dermatofibroma':3,
    'Melanocytic nevi':4,
    'Vascular lesions':5,
    'Melanoma':6,
}

In [ ]:
# Images 

In [ ]:
# load images in 
def load_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, (224, 224))  # Resize to fit model input, lower to 28?
        images.append(img)
    return np.array(images)

# Assuming you have a column with image paths
images = load_images(df['image_path'].values)
labels = df['label'].values 

In [ ]:
# Split the DataSet

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.05, random_state=42)

In [ ]:
# Build CNN (conv neural network) Model 

In [ ]:
def build_CNN_model(input_shape=(224, 224, 3), num_classes=1000):
    model = models.Sequential()

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)), 
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    
    Conv2D(512,(3,3),input_shape=(28,28,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    
    Dropout(0.3),
    
    Conv2D(1024, (3, 3), input_shape=(28,28,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    
    Dropout(0.3),
                   
    Conv2D(1024,(1,1), input_shape=(28,28,3), activation='relu'),
    MaxPooling2D(pool_size=(1, 1)),
    BatchNormalization(),
    
    Dropout(0.3),
    Conv2D(1024,(1,1),input_shape=(28,28,3),activation='relu'),
    MaxPooling2D(pool_size=(1, 1)),
    BatchNormalization(),

    Dropout(0.3),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax') 
])


In [ ]:
model.compile(optimizer='adam',loss= 'categorical_crossentropy', metrics= ['accuracy'])


In [ ]:
model.summary()

In [ ]:
#Training our CNN Model
class_weights = {0:1,1:0.5,2:1,3:1,4:1,5:1,6:1}
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=90, class_weight=class_weights,validation_data=(y_train, y_test))

In [ ]:
#Evaluate the Model 
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
# Make Predictions
predictions=model.predict_classes(y_train)

check=[]
for i in range(len(y_test)):
  for j in range(7):
    if(y_test[i][j]==1):
      check.append(j)
check=np.asarray(check)

print(classification_report(check, predictions))

In [ ]:
# model.save('modelv1.h5')

In [ ]:
# Check accuracy
from sklearn.metrics import accuracy_score
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")